## Model extraction and reproduction

### Extract parameters from a .pt file

In [1]:
import os
import torch
import random
import numpy as np
def seed_everything(seed=20):
    """set seed for all"""
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [2]:
# load the model
import torch
from torch import nn
sent_model = torch.load('saved_models/A_model_1.pt', map_location='cpu')
sent_model.eval()
for param in sent_model.parameters():
    param.requires_grad = False
print(sent_model)

Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=128, bias=True)
  (2): GELU()
  (3): Linear(in_features=128, out_features=3, bias=True)
)


In [3]:
# get model structure
sent_model_structure = '|'.join([layer_str.split('): ')[1] for layer_str in str(sent_model).split('\n')[1:-1]])
print('\n'.join(sent_model_structure.split('|')))

BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Linear(in_features=768, out_features=128, bias=True)
GELU()
Linear(in_features=128, out_features=3, bias=True)


In [4]:
# get model parameters
with torch.no_grad():
    sent_model_parameters = {name:val.detach() for name, val in sent_model.named_parameters()}

In [5]:
# Since the model uses normalization, we also need to send the mean and variance
sent_model_parameters['0.running_mean'] = sent_model[0].running_mean
sent_model_parameters['0.running_var'] = sent_model[0].running_var

In [6]:
# print weight name and shape
sent_model_parameters_names = list(sent_model_parameters.keys())
sent_model_parameters_vals = list(sent_model_parameters.values())
for name, val in zip(sent_model_parameters_names, sent_model_parameters_vals):
    print(f'{name:8s}\t{val.__class__.__name__}\t{list(val.shape)}'.expandtabs(2))


0.weight  Tensor  [768]
0.bias    Tensor  [768]
1.weight  Tensor  [128, 768]
1.bias    Tensor  [128]
3.weight  Tensor  [3, 128]
3.bias    Tensor  [3]
0.running_mean  Tensor  [768]
0.running_var Tensor  [768]


### Test Reproductivity

In [7]:
# We will send 3 parameters to a client
'''
    sent_model_structure
    sent_model_parameters_names
    sent_model_parameters_vals
''';

import base
uploader = base.login("info@openmined.org", "changethis")

reciever = base.login("sheldon@caltech.edu","bazinga")


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into test_domain... done!
Connecting to localhost... done! 	 Logging into test_domain... done!


In [8]:
uploader.privacy_budget
uploader.datasets


Idx,Name,Description,Assets,Id
[0],COVID19 Cases in 175 countries,Weekly data for an entire year,"[""Country 0""] -> Tensor[""Country 1""] -> Tensor[""Country 2""] -> Tensor...",5a09e9d4-6c2b-4679-b773-bd9efcff17f4
[1],model parameter,"BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)|Linear(in_features=768, out_features=128, bias=True)|GELU()|Linear(in_features=128, out_features=3, bias=True)","[""0.weight""] -> Tensor[""0.bias""] -> Tensor[""1.weight""] -> Tensor...",c79399f6-172d-4a3c-93dd-ba790f99a332


In [9]:
#base.upload(uploader,sent_model_parameters_names,sent_model_parameters,sent_model_structure)
#del uploader.datasets[1]


In [10]:
reciever.datasets[1]


Dataset: model parameter
Description: BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)|Linear(in_features=768, out_features=128, bias=True)|GELU()|Linear(in_features=128, out_features=3, bias=True)



Asset Key,Type,Shape
"[""0.weight""]",Tensor,"(768,)"
"[""0.bias""]",Tensor,"(768,)"
"[""1.weight""]",Tensor,"(128, 768)"
"[""1.bias""]",Tensor,"(128,)"
"[""3.weight""]",Tensor,"(3, 128)"
"[""3.bias""]",Tensor,"(3,)"
"[""0.running_mean""]",Tensor,"(768,)"
"[""0.running_var""]",Tensor,"(768,)"


In [11]:
# Stimulate a client who receives the parameter
received_model_structure= reciever.datasets[1].description


In [12]:
# initialize the model
received_model = nn.Sequential(
    *[eval('nn.' + layer) for layer in received_model_structure.split('|')]
)
# print part of the initial model weight
print(received_model[0].weight.detach()[:10])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [13]:
# request for weight
sent=reciever.datasets[1]
asset= sent.assets
for i in asset:
    result = sent[i['name']]
    result.request(reason='research')


In [14]:
# accept all request
for i in uploader.requests:
    i.accept()

In [15]:
for i in asset:
    name=i['name']
    result= sent[name]
    name=f'[{name.split(".")[0]}].{name.split(".")[1]}'
    x=result.get().child.child
    x=torch.tensor(x).to(dtype=torch.float32)
    exec(f'received_model{name} = nn.Parameter(x)')

In [16]:
# make sure the weight has been changed
print(received_model[0].weight.detach()[:10])


tensor([0.9913, 1.0620, 1.0096, 0.9389, 1.0044, 1.0548, 0.9228, 0.9387, 1.0055,
        0.9827])


### Make sure two models have the same output for a given input

In [17]:
# Random input for the model
random_input = torch.randn(32, 768)

In [18]:
# test the output of sent_model
sent_model.eval()
for param in sent_model.parameters():
    param.requires_grad = False
sent_model_output = sent_model(random_input)
sent_model_output[:5]

tensor([[ 0.6840, -0.6742, -0.8874],
        [ 0.4475, -0.3338, -1.1385],
        [ 0.6013, -0.7323, -0.7802],
        [ 0.4355, -0.3431, -0.8839],
        [ 0.5737, -0.6191, -0.7754]])

In [19]:
# test the output of received_model
received_model.eval()
for param in received_model.parameters():
    param.requires_grad = False
received_model_output = received_model(random_input)
received_model_output[:5]

tensor([[ 0.6840, -0.6742, -0.8874],
        [ 0.4475, -0.3338, -1.1385],
        [ 0.6013, -0.7323, -0.7802],
        [ 0.4355, -0.3431, -0.8839],
        [ 0.5737, -0.6191, -0.7754]])

In [20]:
if (sent_model_output == received_model_output).all():
    print('Success! Two models have the same output')
else:
    print('Fail! Please check if any of the parameters are different')

Success! Two models have the same output
